In [124]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [131]:
import numpy as np

from jaccard import *

In [62]:
np.set_printoptions(precision=4, suppress=True, floatmode='fixed')

In [135]:
V = np.array([
    [1, 1, 0],
    [1, 0, 0],
])
C = np.array([0.5, 0.5, 0])

In [167]:
C = np.array([0.5, 0.5, 0])
for i in range(5):
    print(f"C={C}")
    C = gradient_descend(C, V, 1, 0.4, 'max')
print(f"C={C}")

C=[0.5000 0.5000 0.0000]
C=[0.7561 0.4570 0.0000]
C=[1.0000 0.4152 0.0000]
C=[1.0000 0.4144 0.0000]
C=[1.0000 0.4142 0.0000]
C=[1.0000 0.4142 0.0000]


In [71]:
dist = jaccard(V, C.reshape(1, -1))
grad = gradient(C, V)
edist = np.e ** dist
softmax = edist / edist.sum()
print(dist)
print(softmax)
print(grad)
print(dist * grad)
print(softmax * grad)

[[0.0255]
 [0.4869]]
[[0.3866]
 [0.6134]]
[[ 0.0000  0.5000  0.0000]
 [ 0.0000 -0.2632  0.0000]]
[[ 0.0000  0.0127  0.0000]
 [ 0.0000 -0.1282  0.0000]]
[[ 0.0000  0.1933  0.0000]
 [ 0.0000 -0.1615  0.0000]]


[[18.7074]
 [18.7078]]
[[ 0.0000  0.5000  0.0000]
 [ 0.0000 -0.5000  0.0000]]
[[ 0.0000  0.2500  0.0000]
 [ 0.0000 -0.2500  0.0000]]
